# Convert — Run a simple conversion of a PDB to the QDXF format

This notebook shows how to perform a simple conversion of PDBs to the QDXF format

# 0) Complete example
See the [sample notebook](/Quickstarts/convert-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import os
from glob import glob
from pathlib import Path

from pdbtools import pdb_fetch

import rush

## 1.1) Configuration

In [ ]:
# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "convert"]
WORK_DIR = Path.home() / "qdx" / "convert"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

In [ ]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

In [ ]:
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## 1.2) Build your client

Initialize our rush client and fetch available module paths.

In [ ]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR, batch_tags=TAGS, restore_by_default=True
)

# 2) Fetch PDBS

In [ ]:
pdb_ids = ["1B39", "1GIH"]
for pdb_id in pdb_ids:
    complex = list(pdb_fetch.fetch_structure(pdb_id))
    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb_id}.pdb", "w") as f:
        for line in complex:
            f.write(line)

# 3) Convert each PDB

In [ ]:
for pdb in glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")):
    (conformer,) = await client.convert("PDB", Path(pdb), target="NIX")
    print(await conformer.get())

2024-02-28 18:33:24,610 - rush - INFO - Trying to restore job with tags: ['qdx', 'rush-py-v2', 'demo', 'convert'] and path: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#convert
2024-02-28 18:33:24,920 - rush - INFO - Argument dc76b276-72e5-4f58-bd4b-411af1650e40 is now ModuleInstanceStatus.RESOLVING
2024-02-28 18:33:26,036 - rush - INFO - Argument dc76b276-72e5-4f58-bd4b-411af1650e40 is now ModuleInstanceStatus.RUNNING
2024-02-28 18:33:27,246 - rush - INFO - Argument dc76b276-72e5-4f58-bd4b-411af1650e40 is now ModuleInstanceStatus.AWAITING_UPLOAD
url='https://storage.googleapis.com/rush_store_default/d4a18db2-b727-4f5d-b802-4b5c9c8969d6?x-goog-signature=051ec7adc46c46f15da0e8460b1f1cc35e7418d02df0fb5f2f2a6fb237796714a51bb36b15a87625042f28b1d0953b3e4e587e5fca0a679a3f21696d0c0cda68275b4a320c74acab2b48057df7c162c75b5d9726dd5b8632e5e66c0dc6e7336c988123309106bbb7a1c58871329d9b15dee2d58ae6c04fc45e81e906e3a968bb39f661828fe213c3cba62e043001a98d9d0cbb053eab60276b257d519524

In [ ]:
await client.status()

{'c3156fa9-9a73-4639-9226-67e3c64682df': (<ModuleInstanceStatus.COMPLETED: 'COMPLETED'>,
  'convert',
  1)}